In [1]:
import pandas as pd

In [2]:
data_home = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/data_home"

# Map affy IDs to rs IDs

In [5]:
df = pd.read_csv("%s/Axiom_tx_v1.na35.annot.csv" % data_home,skiprows=19)

/home/hv270/platelets/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df["AX"] = "AX-" + df["Affy SNP ID"].str.split("-",expand=True)[1]

In [43]:
df.iloc[100000]

Probe Set ID                                                             AX-11601959
Affy SNP ID                                                            Affx-33495517
dbSNP RS ID                                                                rs7021669
dbSNP Loctype                                                                      2
Chromosome                                                                         9
Physical Position                                                           19468614
Position End                                                                19468614
Strand                                                                             +
ChrX pseudo-autosomal region 1                                                     0
Cytoband                                                                       p22.1
Flank                              CTTCTTTTTTGTAGATCTAGGCTATATATCACATT[G/T]CATTAG...
Allele A                                                         

In [25]:
interesting_snps = [
    "rs12566888", #PEAR1
]

In [44]:
df.loc[df["dbSNP RS ID"].isin(interesting_snps),"Probe Set ID"].tolist()

['AX-39102413']

In [36]:
snps = pd.read_csv("%s/Genomes/plink.snplist" % data_home,header=None)[0].tolist()

In [40]:
import numpy as np

In [45]:
"AX-39102413" in snps

True

In [46]:
#~/plink --bfile Genomes/filtered_S159 --snp AX-39102413 --recode tab --out Genomes/S159_PEAR1

In [73]:
pear1 = pd.read_csv("%s/Genomes/S159_PEAR1.ped" % data_home,sep="\t",header=None)

In [74]:
pear1["PEAR1"] = 0*(pear1[6] == "G G") + 1*(pear1[6] == "T G") + 2*(pear1[6] == "T T")

In [76]:
pear1 = pear1[[1,"PEAR1"]]

In [77]:
IDs = pd.read_excel("%s/Genomes/CBR159_IDs.xlsx" % data_home)

/home/hv270/platelets/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


In [86]:
df = pd.merge(IDs,pear1,left_on="CBR ID",right_on=1,
         how="inner")[["CFT 4R ID","PEAR1"]].rename(columns={"CFT 4R ID":"ID"})

In [87]:
df.to_csv("%s/PEAR1_PFC.csv" % data_home)

# Interval cohort

In [89]:
#~/home/hv270/plink --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1 --snp rs12566888 --out PEAR1_INTERVAL --recode tab

In [93]:
pear1 = pd.read_csv("%s/Genomes/PEAR1_INTERVAL.ped" % data_home,sep="\t",header=None)
pear1["PEAR1"] = 0*(pear1[6] == "G G") + 1*(pear1[6] == "T G") + 2*(pear1[6] == "T T")

In [96]:
matching = pd.read_csv("/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_phenotypes/INTERVAL_OmicsMap_20200514.csv", sep=",")
has_one_id = ~matching["Affymetrix_QC_bl"].isnull() | ~matching["Affymetrix_QC_24m"].isnull()
matching = matching.loc[has_one_id]
matching["AffyID"] = matching["Affymetrix_QC_bl"]
matching.loc[matching.AffyID.isnull(),"AffyID"] = matching.loc[matching.AffyID.isnull(),"Affymetrix_QC_24m"]

In [108]:
from prediction_pipeline.preprocessing import *

/home/hv270/platelets/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


In [109]:
int_phen, meta = get_INTERVAL_phenotypes()
meta = pd.merge(matching,int_phen,left_on="identifier", right_on="identifier")
meta["IID"] = meta["AffyID"].astype(int).astype(str)
meta["FID"] = meta["AffyID"].astype(int).astype(str)
meta["cohort"] = meta.interval

Loading INTERVAL phenotypes
(48460, 760)
(45040, 760)
(38832, 760)
(112535, 11)
(102595, 11)
(17768, 772)
(32574, 777)
(32727, 777)
Checked that each Sample has one row max
Removing samples with less than 10 measurements per day
(30725, 782)


In [118]:
meta.iloc[0]

identifier                      1000003
Affymetrix_QC_bl         110012593138.0
Affymetrix_gwasQC_bl     110012593138.0
Affymetrix_QC_24m                   NaN
Affymetrix_gwasQC_24m               NaN
                              ...      
yearday                             183
hours                                10
IID                        110012593138
FID                        110012593138
cohort                              F14
Name: 0, Length: 790, dtype: object

In [122]:
df = pd.merge(meta,pear1[[0,"PEAR1"]],left_on="AffyID",right_on=0,how="right")[["SampleNo","PEAR1"]]
df = df.loc[~df.SampleNo.isnull()]

In [124]:
df.to_csv("%s/PEAR1_INTERVAL.csv" % data_home)

# Get PCs for Interval

In [126]:
export_folder = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing"

In [127]:
pcs = pd.read_csv("%s/principal_components/PC.eigenvec" % export_folder,sep="\t")
pcs["IID"] = pcs["IID"].astype(str)

In [131]:
df = pd.merge(pcs,meta, left_on="IID",right_on="IID",how="inner")[["SampleNo"] + list(pcs.columns[2:])]

In [132]:
df.to_csv("%s/PCs_INTERVAL.csv" % data_home)